# Spotify Data Cleaning & Processing

This notebook prepares the three raw datasets for visualization:
- `charts.csv` (daily regional charts)
- `Most Streamed Spotify Songs 2024.csv` (cross-platform performance)
- `tracks.csv` (audio features & genres)


## 1. Load raw datasets

In this section we load the three CSV files and do very light inspection (shapes, columns, sample rows). We'll add cleaning/processing in later sections.

In [20]:
%pip install pandas
import os
from pathlib import Path

import pandas as pd

PROJECT_ROOT = Path(r"C:/Users/Muhammad Bilal/Desktop/dataviz 1")
DATA_DIR = PROJECT_ROOT / "viz" / "data" / "raw"
PROCESSED_DIR = PROJECT_ROOT / "viz" / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print("DATA_DIR:", DATA_DIR)
print("Exists:", DATA_DIR.exists())
print("Contents:", list(DATA_DIR.iterdir()))
print("PROCESSED_DIR:", PROCESSED_DIR)

# Helper to read CSVs with basic options

def read_csv_safe(path: Path, **kwargs) -> pd.DataFrame:
    print(f"Reading {path} ...")
    return pd.read_csv(path, low_memory=False, **kwargs)

charts_path = DATA_DIR / "charts.csv" 
most_streamed_path = DATA_DIR / "Most Streamed Spotify Songs 2024.csv"
tracks_path = DATA_DIR / "tracks.csv"

charts_head = read_csv_safe(charts_path, nrows=5)
most_streamed_head = read_csv_safe(most_streamed_path, nrows=5, encoding="latin1", encoding_errors="replace")
tracks_head = read_csv_safe(tracks_path, nrows=5)

charts_head, most_streamed_head, tracks_head

Note: you may need to restart the kernel to use updated packages.
DATA_DIR: C:\Users\Muhammad Bilal\Desktop\dataviz 1\viz\data\raw
Exists: True
Contents: [WindowsPath('C:/Users/Muhammad Bilal/Desktop/dataviz 1/viz/data/raw/charts.csv'), WindowsPath('C:/Users/Muhammad Bilal/Desktop/dataviz 1/viz/data/raw/Most Streamed Spotify Songs 2024.csv'), WindowsPath('C:/Users/Muhammad Bilal/Desktop/dataviz 1/viz/data/raw/tracks.csv')]
PROCESSED_DIR: C:\Users\Muhammad Bilal\Desktop\dataviz 1\viz\data\processed
Reading C:\Users\Muhammad Bilal\Desktop\dataviz 1\viz\data\raw\charts.csv ...
Reading C:\Users\Muhammad Bilal\Desktop\dataviz 1\viz\data\raw\Most Streamed Spotify Songs 2024.csv ...
Reading C:\Users\Muhammad Bilal\Desktop\dataviz 1\viz\data\raw\tracks.csv ...



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


(                         title  rank        date  \
 0      Chantaje (feat. Maluma)     1  2017-01-01   
 1  Vente Pa' Ca (feat. Maluma)     2  2017-01-01   
 2   Reggaetón Lento (Bailemos)     3  2017-01-01   
 3                       Safari     4  2017-01-01   
 4                  Shaky Shaky     5  2017-01-01   
 
                                   artist  \
 0                                Shakira   
 1                           Ricky Martin   
 2                                   CNCO   
 3  J Balvin, Pharrell Williams, BIA, Sky   
 4                           Daddy Yankee   
 
                                                  url     region   chart  \
 0  https://open.spotify.com/track/6mICuAdrwEjh6Y6...  Argentina  top200   
 1  https://open.spotify.com/track/7DM4BPaS7uofFul...  Argentina  top200   
 2  https://open.spotify.com/track/3AEZUABDXNtecAO...  Argentina  top200   
 3  https://open.spotify.com/track/6rQSrBHf7HlZjtc...  Argentina  top200   
 4  https://open.spotify.com

## 2. Cleaning plan (high level)

We'll implement the cleaning/processing in later cells, roughly:

- **charts.csv**: parse dates, ensure numeric `rank`/`streams`, possibly filter by years/regions, and aggregate for chart-friendly tables.
- **Most Streamed Spotify Songs 2024.csv**: coerce numeric columns (remove commas), parse `Release Date`, and create tidy tables for rankings and platform mixes.
- **tracks.csv**: coerce booleans (`explicit`), ensure numeric audio features, and prepare per-genre summaries and feature matrices.

We'll also consider writing out smaller, analysis-ready files (e.g. `processed_charts.parquet`, `processed_most_streamed.parquet`, `processed_tracks.parquet`).

## 3. `charts.csv` cleaning

Goals:
- Parse the `date` column.
- Ensure `rank` and `streams` are numeric.
- Optionally filter to a max rank (e.g. top 50) and a minimum date to keep file sizes manageable.
- Write out a processed CSV we can load quickly for visualizations.

In [21]:
def clean_charts(
    in_path: Path = charts_path,
    out_path: Path | None = None,
    max_rank: int | None = 50,
    min_date: str | None = None,
    chunksize: int = 250_000,
) -> Path:
    """Stream-clean charts.csv into a smaller, tidy CSV.

    - Reads in chunks to avoid loading the 3GB file into memory.
    - Parses dates and numeric fields.
    - Optionally keeps only rows with rank <= max_rank and date >= min_date.
    """

    if out_path is None:
        out_path = PROCESSED_DIR / "charts_clean_top50.csv"

    # Remove existing file if present so we can append cleanly
    if out_path.exists():
        out_path.unlink()

    parse_dates = ["date"]

    for chunk in pd.read_csv(in_path, chunksize=chunksize, parse_dates=parse_dates):
        # Ensure dtypes
        chunk["rank"] = pd.to_numeric(chunk["rank"], errors="coerce")
        chunk["streams"] = pd.to_numeric(chunk["streams"], errors="coerce")

        if min_date is not None:
            chunk = chunk[chunk["date"] >= pd.to_datetime(min_date)]

        if max_rank is not None:
            chunk = chunk[chunk["rank"] <= max_rank]

        # Drop rows that failed conversion
        chunk = chunk.dropna(subset=["rank", "streams", "date"])

        # Append to output CSV
        header = not out_path.exists()
        chunk.to_csv(out_path, mode="a", index=False, header=header)

    print("Wrote cleaned charts to", out_path)
    return out_path


clean_charts()

Wrote cleaned charts to C:\Users\Muhammad Bilal\Desktop\dataviz 1\viz\data\processed\charts_clean_top50.csv


WindowsPath('C:/Users/Muhammad Bilal/Desktop/dataviz 1/viz/data/processed/charts_clean_top50.csv')

## 4. `Most Streamed Spotify Songs 2024.csv` cleaning

Goals:
- Handle non-UTF8 encoding.
- Parse `Release Date` as a proper datetime.
- Convert key numeric columns with commas (e.g. `Spotify Streams`) into numeric types.
- Save a compact, analysis-ready version.

In [23]:
def clean_most_streamed(in_path: Path = most_streamed_path, out_stem: str = "most_streamed_2024_clean") -> tuple[Path, Path | None]:
    """Clean the 2024 most-streamed dataset and write CSV (+ optional parquet).

    - Uses latin1 + encoding_errors="replace" to avoid Unicode issues.
    - Parses release dates.
    - Converts selected numeric columns that contain commas into real numbers.
    """

    df = pd.read_csv(in_path, encoding="latin1", encoding_errors="replace", low_memory=False)

    # Parse release date
    if "Release Date" in df.columns:
        df["Release Date"] = pd.to_datetime(df["Release Date"], errors="coerce")

    # Columns with large numeric values often stored as strings with commas
    numeric_like_cols = [
        "Spotify Streams",
        "Spotify Playlist Count",
        "Spotify Playlist Reach",
        "Spotify Popularity",
        "YouTube Views",
        "YouTube Likes",
        "TikTok Posts",
        "TikTok Likes",
        "TikTok Views",
        "Apple Music Playlist Count",
        "AirPlay Spins",
        "Deezer Playlist Count",
        "Deezer Playlist Reach",
        "Pandora Streams",
        "Pandora Track Stations",
        "Soundcloud Streams",
        "Shazam Counts",
    ]

    for col in numeric_like_cols:
        if col in df.columns:
            df[col] = (
                df[col]
                .astype(str)
                .str.replace(",", "", regex=False)
                .replace({"": pd.NA})
            )
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Write outputs
    out_csv = PROCESSED_DIR / f"{out_stem}.csv"
    df.to_csv(out_csv, index=False)

    out_parquet = PROCESSED_DIR / f"{out_stem}.parquet"
    try:
        df.to_parquet(out_parquet, index=False)
    except Exception as e:  # pyarrow/fastparquet may not be installed
        print("Could not write parquet (install pyarrow or fastparquet if needed):", e)
        out_parquet = None

    print("Wrote cleaned most-streamed data to", out_csv)
    return out_csv, out_parquet


clean_most_streamed()

Could not write parquet (install pyarrow or fastparquet if needed): Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.
Wrote cleaned most-streamed data to C:\Users\Muhammad Bilal\Desktop\dataviz 1\viz\data\processed\most_streamed_2024_clean.csv


(WindowsPath('C:/Users/Muhammad Bilal/Desktop/dataviz 1/viz/data/processed/most_streamed_2024_clean.csv'),
 None)

## 5. `tracks.csv` cleaning

Goals:
- Ensure audio feature columns are numeric.
- Coerce `explicit` to a proper boolean.
- Save a clean version for feature-space analysis and joining with other datasets.

In [24]:
def clean_tracks(in_path: Path = tracks_path, out_stem: str = "tracks_clean") -> tuple[Path, Path | None]:
    """Clean the audio-features dataset and write CSV (+ optional parquet).

    - Ensures numeric dtypes for audio features and popularity.
    - Coerces the `explicit` flag to boolean.
    """

    df = pd.read_csv(in_path, low_memory=False)

    # Explicit to boolean
    if "explicit" in df.columns:
        df["explicit"] = df["explicit"].astype(str).str.lower().isin(["true", "1", "yes"])

    numeric_cols = [
        "popularity",
        "duration_ms",
        "danceability",
        "energy",
        "key",
        "loudness",
        "mode",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "liveness",
        "valence",
        "tempo",
        "time_signature",
    ]

    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    out_csv = PROCESSED_DIR / f"{out_stem}.csv"
    df.to_csv(out_csv, index=False)

    out_parquet = PROCESSED_DIR / f"{out_stem}.parquet"
    try:
        df.to_parquet(out_parquet, index=False)
    except Exception as e:
        print("Could not write parquet (install pyarrow or fastparquet if needed):", e)
        out_parquet = None

    print("Wrote cleaned tracks data to", out_csv)
    return out_csv, out_parquet


clean_tracks()

Could not write parquet (install pyarrow or fastparquet if needed): Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.
Wrote cleaned tracks data to C:\Users\Muhammad Bilal\Desktop\dataviz 1\viz\data\processed\tracks_clean.csv


(WindowsPath('C:/Users/Muhammad Bilal/Desktop/dataviz 1/viz/data/processed/tracks_clean.csv'),
 None)

In [5]:
# Overwrite charts_clean_top50.csv with top 50 entries from most_streamed_2024_clean

from pathlib import Path
import pandas as pd

# Set up paths
PROJECT_ROOT = Path(r"C:/Users/Muhammad Bilal/Desktop/dataviz 1")
PROCESSED_DIR = PROJECT_ROOT / "viz" / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Load the cleaned 2024 most-streamed dataset
ms_clean = pd.read_csv(PROCESSED_DIR / "most_streamed_2024_clean.csv")

# Ensure we have a ranking column to sort by (fallback to Spotify Streams if needed)
rank_col = "All Time Rank" if "All Time Rank" in ms_clean.columns else None

if rank_col is not None:
    # Convert All Time Rank to numeric (handles strings with commas like "1,000")
    ms_clean[rank_col] = (
        ms_clean[rank_col]
        .astype(str)
        .str.replace(",", "", regex=False)
        .replace({"": pd.NA, "nan": pd.NA})
    )
    ms_clean[rank_col] = pd.to_numeric(ms_clean[rank_col], errors="coerce")
    
    # Sort by numeric rank and take top 50
    top50 = ms_clean.sort_values(rank_col, ascending=True, na_position="last").head(50)
else:
    # Fallback: use Spotify Streams descending if rank column is missing
    top50 = ms_clean.sort_values("Spotify Streams", ascending=False).head(50)

out_path = PROCESSED_DIR / "charts_clean_top50.csv"

# Overwrite the existing file with this new top-50 table
top50.to_csv(out_path, index=False)
print("Replaced charts_clean_top50.csv with top 50 rows from most_streamed_2024_clean →", out_path)


Replaced charts_clean_top50.csv with top 50 rows from most_streamed_2024_clean → C:\Users\Muhammad Bilal\Desktop\dataviz 1\viz\data\processed\charts_clean_top50.csv


## 6. Export data for frontend visualizations

Convert cleaned CSVs to JSON files that can be easily loaded in the Next.js app.


In [11]:
import json
from pathlib import Path
import numpy as np

# Path to public data folder in Next.js app
PUBLIC_DATA_DIR = PROJECT_ROOT / "viz" / "public" / "data"
PUBLIC_DATA_DIR.mkdir(parents=True, exist_ok=True)

# Helper function to clean NaN values for JSON export
def clean_for_json(obj):
    """Recursively replace NaN, NaT, and other non-JSON-serializable values with null"""
    if isinstance(obj, dict):
        return {k: clean_for_json(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [clean_for_json(item) for item in obj]
    elif pd.isna(obj):
        return None
    elif isinstance(obj, float) and np.isnan(obj):
        return None
    elif isinstance(obj, pd.Timestamp):
        return str(obj)
    else:
        return obj

# Export top 50 as JSON for leaderboard
top50_json = top50.to_dict(orient="records")
top50_json_clean = clean_for_json(top50_json)
with open(PUBLIC_DATA_DIR / "top50_tracks.json", "w", encoding="utf-8") as f:
    json.dump(top50_json_clean, f, indent=2)
print(f"Exported top 50 tracks to {PUBLIC_DATA_DIR / 'top50_tracks.json'}")

# Export full most_streamed dataset (for other visualizations)
ms_full = pd.read_csv(PROCESSED_DIR / "most_streamed_2024_clean.csv")
ms_json = ms_full.to_dict(orient="records")
ms_json_clean = clean_for_json(ms_json)
with open(PUBLIC_DATA_DIR / "most_streamed_2024.json", "w", encoding="utf-8") as f:
    json.dump(ms_json_clean, f, indent=2)
print(f"Exported full most_streamed dataset to {PUBLIC_DATA_DIR / 'most_streamed_2024.json'}")

# Export tracks data (sample top 1000 by popularity for now to keep file size manageable)
tracks_df = pd.read_csv(PROCESSED_DIR / "tracks_clean.csv")
tracks_top = tracks_df.nlargest(1000, "popularity")
tracks_json = tracks_top.to_dict(orient="records")
tracks_json_clean = clean_for_json(tracks_json)
with open(PUBLIC_DATA_DIR / "tracks_sample.json", "w", encoding="utf-8") as f:
    json.dump(tracks_json_clean, f, indent=2)
print(f"Exported top 1000 tracks by popularity to {PUBLIC_DATA_DIR / 'tracks_sample.json'}")


Exported top 50 tracks to C:\Users\Muhammad Bilal\Desktop\dataviz 1\viz\public\data\top50_tracks.json
Exported full most_streamed dataset to C:\Users\Muhammad Bilal\Desktop\dataviz 1\viz\public\data\most_streamed_2024.json
Exported top 1000 tracks by popularity to C:\Users\Muhammad Bilal\Desktop\dataviz 1\viz\public\data\tracks_sample.json


## 7. Export Billboard Hot 100 datasets for frontend

Convert the new Billboard datasets to JSON files.


In [2]:
# Export Billboard Hot 100 datasets
from pathlib import Path
import pandas as pd
import json
import numpy as np

# Set up paths
PROJECT_ROOT = Path(r"C:/Users/Muhammad Bilal/Desktop/dataviz 1")
PUBLIC_DATA_DIR = PROJECT_ROOT / "viz" / "public" / "data"
PUBLIC_DATA_DIR.mkdir(parents=True, exist_ok=True)
NEW_DATA_DIR = PROJECT_ROOT / "viz" / "newdata"

# Helper function to clean NaN values for JSON export (same as Cell 12)
def clean_for_json(obj):
    """Recursively replace NaN, NaT, and other non-JSON-serializable values with null"""
    if isinstance(obj, dict):
        return {k: clean_for_json(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [clean_for_json(item) for item in obj]
    elif pd.isna(obj):
        return None
    elif isinstance(obj, float) and np.isnan(obj):
        return None
    elif isinstance(obj, pd.Timestamp):
        return str(obj)
    else:
        return obj

# Export weekly top 50 dataset
weekly_df = pd.read_csv(NEW_DATA_DIR / "option_a_top50_2025.csv")
weekly_df["week_date"] = pd.to_datetime(weekly_df["week_date"], errors="coerce")
weekly_json = weekly_df.to_dict(orient="records")
weekly_json_clean = clean_for_json(weekly_json)
with open(PUBLIC_DATA_DIR / "billboard_weekly_top50.json", "w", encoding="utf-8") as f:
    json.dump(weekly_json_clean, f, indent=2)
print(f"Exported weekly top 50 to {PUBLIC_DATA_DIR / 'billboard_weekly_top50.json'}")

# Export year-end top 50 dataset
yearend_df = pd.read_csv(NEW_DATA_DIR / "derived_yearend_top50_2025.csv")
yearend_json = yearend_df.to_dict(orient="records")
yearend_json_clean = clean_for_json(yearend_json)
with open(PUBLIC_DATA_DIR / "billboard_yearend_top50.json", "w", encoding="utf-8") as f:
    json.dump(yearend_json_clean, f, indent=2)
print(f"Exported year-end top 50 to {PUBLIC_DATA_DIR / 'billboard_yearend_top50.json'}")


Exported weekly top 50 to C:\Users\Muhammad Bilal\Desktop\dataviz 1\viz\public\data\billboard_weekly_top50.json
Exported year-end top 50 to C:\Users\Muhammad Bilal\Desktop\dataviz 1\viz\public\data\billboard_yearend_top50.json
